In [1]:
import numpy as np
from numpy.linalg import inv

In [2]:
x = np.array([94, 96, 94, 95, 104, 106, 108, 113, 115, 121, 131])
y = [0.47, 0.75, 0.83, 0.98, 1.18, 1.29, 1.40, 1.60, 1.75, 1.90, 2.23]
N = len(x)

X = np.hstack((np.ones((N, 1)), x.reshape(N, 1)))
w = inv(X.T.dot(X)).dot(X.T).dot(y)
y_hat = X.dot(w)
sigma_hat2 = sum((y - y_hat) ** 2) / (N - 2.0)
V0_inv = np.array([[0, 0],[0, 1]])
w0 = np.array([1, 0])